## Updates

In [6]:
from IPython.display import Javascript, display

class DialogBox():
    
    def __init__(self, title, body):
        
        self.js = """require(["base/js/dialog"],function(dialog) {{dialog.modal({{title: "{0}",body: "{1}",buttons: {{'OK': {{}}}} }});}});""".format(title, body)
        
        print(self.js)
        
        self.display()
        
    def display(self):
        display(Javascript(self.js))

In [7]:
class QuizTimeOut(Exception):
    def __init__(self, body):
        DialogBox('Quiz Time Out', body)

In [8]:
from datetime import datetime, timedelta

def render(self):

    # if date and time does not => than current time and date display a beautiful message and clear it
    #         datetime1 = datetime.strptime('07/11/2019 02:45PM', '%m/%d/%Y %I:%M%p')
    quiz_time = datetime.strptime(self.start_datetime, '%m/%d/%Y %I:%M%p')
    
    prep_time = 5 # loading time
    
    alloted_time = (quiz_time + timedelta(seconds=(self.alloted_time + prep_time)) -  datetime.now()).total_seconds()
    
    if quiz_time > datetime.today():
        raise QuizTimeOut('There is an active quiz, but please check back at {} prompt!'.format(quiz_time.strftime('%I:%M%p')))
    
    if alloted_time <= 0:
        raise QuizTimeOut('There is not active quiz at this time')

    display(widgets.VBox(self.quiz_container))

    self.timer = threading.Timer(alloted_time, self.callback)
    self.timer.start() 



In [ ]:
#quiz timing with datetime

import threading
import time

class TimerClass(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.event = threading.Event()
        self.count = 10

    def run(self):
        while self.count > 0 and not self.event.is_set():
            print (self.count)
            self.count -= 1
            self.event.wait(1)

    def stop(self):
        self.event.set()

tmr = TimerClass()
tmr.start()

time.sleep(7)

tmr.stop()

package as pip or/and

https://packaging.python.org/tutorials/packaging-projects/

https://marthall.github.io/blog/how-to-package-a-python-app/

use it as jupyter extension

http://jupyter-notebook.readthedocs.io/en/stable/extending/frontend_extensions.html

- write the handler function in the link above to call quiz thus:
    - act = Activity() 
    - act.load('~/activity')
    - act.render()

In [1]:
%%javascript
IPython.load_extensions('/home/imachine/Documents/Jupyter_projects/my-extension/main.js');

<IPython.core.display.Javascript object>

Call the tool a in-class-tool or something more general it will then have sub-menu:
- quiz
- report (give an overview of the performance in attendance and quiz)
- lab

Use Tab widget instead of next buttons

## docker commands

docker rmi -f  $(docker images -f dangling=true -q)

/*
Add this file to $(ipython locate)/nbextensions/gist.js
And load it with:

require(["nbextensions/gist"], function (gist_extension) {
    console.log('gist extension loaded');
    gist_extension.load_ipython_extension();
});

*/
define( function () {
    
    var token_name = "gist_github_token";
    
    // dialog to request GitHub OAuth token
    // I'm not sure it's possible to step through OAuth purely client side,
    // so just ask the user to go create a token manually.
    
    var token_dialog = function () {
        var dialog = $('<div/>').append(
            $("<p/>")
                .html('Enter a <a href="https://github.com/settings/tokens" target="_blank">GitHub personal access token</a>:')
        ).append(
            $("<br/>")
        ).append(
            $('<input/>').attr('type','text').attr('size','40')
        );
        IPython.dialog.modal({
            title: "GitHub OAuth",
            keyboard_manager: IPython.notebook.keyboard_manager,
            body: dialog,
            buttons : {
                "Cancel": {},
                "OK": {
                    class: "btn-primary",
                    click: function () {
                        var token = $(this).find('input').val();
                        localStorage[token_name] = token;
                        gist_notebook();
                    }
                }
            },
            open : function (event, ui) {
                var that = $(this);
                // Upon ENTER, click the OK button.
                that.find('input[type="text"]').keydown(function (event, ui) {
                    if (event.which === 13) {
                        that.find('.btn-primary').first().click();
                        return false;
                    }
                });
                that.find('input[type="text"]').focus().select();
            }
        });
    };
    // get the GitHub token, via cookie or 
    var get_github_token = function () {
        var token = localStorage[token_name];
        if (!token) {
            token_dialog();
            return null;
        }
        return token;
    };

    var gist_notebook = function () {
        if (!IPython.notebook) return;
        var gist_id = IPython.notebook.metadata.gist_id;
        console.log(gist_id);
        var token = get_github_token();
        if (!token) {
            // dialog's are async, so we can't do anything yet.
            // the dialog OK callback will continue the process.
            console.log("waiting for auth dialog");
            return;
        }
        var method = "POST";
        var url = "https://api.github.com/gists";
        if (gist_id) {
            url = url + "/" + gist_id;
            method = "PATCH";
        }
        var filedata = {};
        var nbj = IPython.notebook.toJSON();
        if (nbj.nbformat === undefined) {
            // older IPython doesn't put nbformat in the JSON
            nbj.nbformat = IPython.notebook.nbformat;
        }
        filedata[IPython.notebook.notebook_name] = {content : JSON.stringify(nbj, undefined, 1)};
        var settings = {
            type : method,
            headers : { Authorization: "token " + token },
            data : JSON.stringify({
                public : true,
                files : filedata,
            }),
            success : function (data, status) {
                console.log("gist succeeded: " + data.id);
                IPython.notebook.metadata.gist_id = data.id;
                update_gist_link(data.id);
                IPython.notification_area.get_widget("notebook").set_message("gist succeeded: " + data.id, 1500);
            },
            error : function (jqXHR, status, err) {
                if (true || jqXHR.status == 403) {
                    // authentication failed,
                    // delete the token so that we prompt again next time
                    delete localStorage[token_name];
                }
                alert("Uploading gist failed: " + err);
            }
        };
        $.ajax(url, settings);
    };
    
    var update_gist_link = function(gist_id) {
        if (!IPython.notebook) return;
        
        if (!gist_id) {
            gist_id = IPython.notebook.metadata.gist_id;
        } else {
            IPython.notebook.metadata.gist_id = gist_id;
        }
        if (!gist_id) {
            return;
        }
        var toolbar = IPython.toolbar.element;
        var link = toolbar.find("a#nbviewer");
        if ( ! link.length ) {
            link = $('<a id="nbviewer" target="_blank"/>');
            toolbar.append(
                $('<span id="nbviewer_span"/>').append(link)
            );
        }
    
        link.attr("href", "https://nbviewer.jupyter.org/" + gist_id);
        link.text("https://nbviewer.jupyter.org/" + gist_id);
    };

    var gist_button = function () {
        if (!IPython.toolbar) {
            $([IPython.events]).on("app_initialized.NotebookApp", gist_button);
            return;
        }
        if ($("#gist_notebook").length === 0) {
            IPython.toolbar.add_buttons_group([
                {
                    'label'   : 'gist',
                    'help'    : 'Share notebook as gist',
                    'icon'    : 'fa-share',
                    'callback': gist_notebook,
                    'id'      : 'gist_notebook'
                },
            ]);
        }
        update_gist_link();
    };
    
    var load_ipython_extension = function () {
        gist_button();
        update_gist_link();
        $([IPython.events]).on("notebook_loaded.Notebook", function () {update_gist_link();});
    };
    
    return {
        load_ipython_extension : load_ipython_extension,
    };
    
});
